In [2]:
###### Read file predictions.csv saved in UF folder #######

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

epislon = 1e-20

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_true-y_pred)/np.sum(y_true))
  else:
    return 100*(np.sum(y_true-y_pred)/(np.sum(y_true) + epislon))
  
def mase(y_true, y_pred, y_baseline):
  # Calcula o MAE do modelo
  mae_pred = np.mean(np.abs(y_true - y_pred))
  # Calcula o MAE do modelo baseline Persistent Window (i.e., últimas h observações antes do teste)
  mae_naive = np.mean(np.abs(y_true - y_baseline))
  if mae_naive == 0:
    result = mae_pred/(mae_naive + epislon)
  else:
    result = mae_pred/mae_naive
  return result 

def baseline_persistent_window(series, horizon):
  subsequence = (series[-horizon*2:-horizon]).values
  return subsequence

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID


file_path = './predictions.csv'
data = pd.read_csv(file_path, header=None)
horizon = 12

for i in range(data.shape[0]):
  # Redefine variable assignments
  product = data.iloc[i, 0]  # First cell for product
  state = data.iloc[i, 1]    # Second cell for state
  model = data.iloc[i, 2]
  predictions = data.iloc[i, 3:].tolist()  # Remaining cells for predictions
  predictions_df = pd.DataFrame(predictions, columns=['Predictions'])

  ################ Plataforma de Métricas ##############

  df = pd.read_csv(f"./uf/{product}/mensal_{state}_{product}.csv", header=0, sep=";")
  df = pd.DataFrame(df)

  y_baseline = baseline_persistent_window(df, horizon)

  # Step 1: Sort the DataFrame if needed to ensure the last 12 entries are the latest ones
  df_sorted = df.sort_values(by='timestamp', ascending=True)
  df_sorted['timestamp'] = pd.to_datetime(df_sorted['timestamp'].astype(str), format='%Y%m')
  # Step 2: Select the last 12 entries
  last_12_date = df['timestamp'].tail(12)
  last_12_m3 = df['m3'].tail(12)
  # Step 3: Create a new DataFrame with the last 12 'm3' values and assign 'product' and 'estado'
  new_df = pd.DataFrame({
              'date': last_12_date,
              'product': product,
              'state': state,
              'm3': last_12_m3.values})
  new_df = new_df.reset_index(drop=True)

  predictions_float = [float(x) for x in predictions]  # Convert each string in the list to a float

  # Create the DataFrame with float values
  predictions_df = pd.DataFrame(predictions_float, columns=['Predictions'])

  mape_result = mape(new_df['m3'], predictions_df['Predictions'])
  pocid_result = pocid(new_df['m3'], predictions_df['Predictions'])
  pbe_result = pbe(new_df['m3'], predictions_df['Predictions'])
  mase_result = mase(new_df['m3'], predictions_df['Predictions'],y_baseline[:,1])

  ############### Métricas ##############

  print(f"MODELO: {model}, MAPE: {mape_result:.5f}, POCID: {pocid_result:.5f}, PBE: {pbe_result:.5f}, MASE: {mase_result:.5f}")

  # ############### Plot Predictions ##############

  # import matplotlib.pyplot as plt
  # import pandas as pd  # If needed to simulate DataFrame creation

  # # Creating the plot
  # plt.figure(figsize=(6, 3))  # Set the figure size

  # # Plotting future_forecast
  # plt.plot(predictions_df['Predictions'], label='Forecast', color='red', linestyle='--')

  # # Plotting monthly_data_norm_last
  # plt.plot(new_df['m3'], label='Valores Reais', color='blue')

  # # Adding titles and labels
  # plt.title(f'Modelo: {model}, {product} no {state}')
  # plt.xlabel('Meses')
  # plt.ylabel('Valores')

  # # Showing the legend
  # plt.legend()

  # # Displaying the plot
  # plt.show()


MODELO: kNN, MAPE: 0.15656, POCID: 63.63636, PBE: 14.94620, MASE: 0.39517
MODELO: MLP, MAPE: 0.13122, POCID: 45.45455, PBE: -0.72194, MASE: 0.39217
MODELO: Prophet, MAPE: 0.15615, POCID: 90.90909, PBE: 15.86473, MASE: 0.55726
MODELO: RF, MAPE: 0.20470, POCID: 54.54545, PBE: 14.68460, MASE: 0.42365
MODELO: LR, MAPE: 0.09481, POCID: 72.72727, PBE: 7.94454, MASE: 0.30449
MODELO: kNN, MAPE: 0.12556, POCID: 54.54545, PBE: 5.13407, MASE: 0.38192
MODELO: MLP, MAPE: 0.33886, POCID: 63.63636, PBE: 29.92724, MASE: 0.52925
MODELO: XGB, MAPE: 0.26974, POCID: 45.45455, PBE: 27.47075, MASE: 0.61975
MODELO: SVR, MAPE: 0.05862, POCID: 81.81818, PBE: 1.86753, MASE: 0.27221
MODELO: SVR, MAPE: 0.14626, POCID: 45.45455, PBE: -9.69596, MASE: 0.28243
MODELO: XGB, MAPE: 0.10933, POCID: 63.63636, PBE: -0.65227, MASE: 0.29807
MODELO: MLP, MAPE: 0.20491, POCID: 81.81818, PBE: 17.11834, MASE: 0.49195
MODELO: kNN, MAPE: 0.06734, POCID: 45.45455, PBE: 4.13930, MASE: 0.52919
MODELO: XGB, MAPE: 0.22537, POCID: 63.63